In [6]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import re
import itertools
import os
import os.path
from Bio import motifs
from Bio import SeqIO
from Bio.Alphabet import IUPAC
from Bio.Seq import Seq
%matplotlib inline

ImportError: No module named Bio

In [2]:
# get ENCODE TF binding sites
tfbs = pd.read_table('c:/ravip/encode/wgEncodeRegTfbsClusteredV3.bed.gz',
                     header=None,names=['seqname','start','end','gene','score', 'nexpt', 'expts', 'expscores'],
                     dtype={'start':np.int, 'end':np.int})
tfbs[:5]

,seqname,start,end,gene,score,nexpt,expts,expscores
0,chr1,10073,10329,ZBTB33,354,2,"204,246","354,138"
1,chr1,10149,10413,CEBPB,201,1,343,201
2,chr1,16110,16390,CTCF,227,7,"213,612,621,627,628,631,662","110,139,171,209,227,200,170"
3,chr1,29198,29688,TAF1,184,1,157,184
4,chr1,29275,29591,GABPA,198,1,180,198


In [3]:
cdkn2a = pd.read_table('data/cdkn2a.tsv')
cdkn2a[:5]

,RNAFile,DNAFile,Hugo_Symbol,Entrez_Gene_Id,Center,NCBI_Build,Chromosome,Start_Position,End_Position,Strand,...,tumorRNAFraction,tumorDNAMultiple,tumorRNAMultiple,tumorDNARatio,tumorRNARatio,FractionOfMeanExpression,zOfmeanExpression,IsSingle,CancerType,gender
0,hnsc\01510fa3-1918-45a8-836e-6a1b009b53e7-CDKN...,hnsc\0811a03f-37c3-480d-a207-058f2fd784fb-CDKN...,CDKN2A,1029,broad.mit.edu,37,9,21971120,21971120,+,...,0.660000,1.727273,1.941176,1.727273,1.941176,0.224637,-0.522650,True,hnsc,male
1,lusc\01e3b5df-7733-445a-aaa4-ce0809f04736-CDKN...,lusc\904a0fce-3a41-4716-aa0b-5455f82d8e62-CDKN...,CDKN2A,1029,broad.mit.edu,37,9,21971017,21971017,+,...,0.886313,2.083333,7.796053,2.083333,7.796053,2.165528,1.007740,True,lusc,male
2,coad\1c3158f8-0ab0-4205-b344-c39b060550ca-CDKN...,coad\9b09830c-ee71-440e-acd4-c75a3bc76e5a-CDKN...,CDKN2A,1029,hgsc.bcm.edu,37,9,21971058,21971058,+,...,0.851852,-1.555556,5.750000,0.642857,5.750000,2.084673,0.772367,True,other,female
3,hnsc\0a250cb6-7a84-4924-86ee-623793b6abac-CDKN...,hnsc\72cf828e-8158-43da-adee-f6d2e1b0d876-CDKN...,CDKN2A,1029,broad.mit.edu,37,9,21971035,21971035,+,...,0.975845,5.000000,40.400000,5.000000,40.400000,2.162274,0.783455,True,hnsc,female
4,hnsc\0b6f58df-8922-4ba4-9614-eb2bdf40d2f9-CDKN...,hnsc\6662c862-34be-48e2-9823-066d2c239cfd-CDKN...,CDKN2A,1029,broad.mit.edu,37,9,21971029,21971029,+,...,0.887218,1.958333,7.866667,1.958333,7.866667,0.533013,-0.314783,True,hnsc,male


In [3]:
p53 = pd.read_table('data/tp53.tsv')
p53[:5]
muts = p53

In [4]:
def read_vcf(filename, nrows=None, somatic=False, donor=None, skipcols=None):
    names=['chrom','pos','id','ref','alt','qual','filter','info']
    if somatic:
        names = names + ['format','normal','tumor']
    usecols = None if skipcols==None else [i for i in range(len(names)) if names[i] not in skipcols]
    alldna = pd.read_table(filename, names=names,
                    dtype={'chrom':np.character}, comment='#', index_col=False, nrows=nrows, usecols=usecols)
    if not alldna.iloc[0].chrom.startswith('chr'):
        alldna.chrom = alldna.chrom.apply(lambda c: 'chr'+c)
    if donor != None:
        alldna['donor'] = donor
    return alldna

In [72]:
# not useful - only has frequencies, not mapping to individual donors
simple = read_vcf('d:/sequence/pancan/simple_somatic_mutation.aggregated.vcf.gz', nrows=10, donor='ravi')
simple[:5]

,chrom,pos,id,ref,alt,qual,filter,info,donor
0,chr1,10144,MU29575239,TA,T,.,.,"CONSEQUENCE=||||||intergenic_region||,WASH7P|E...",ravi
1,chr1,10151,MU15890267,T,A,.,.,"CONSEQUENCE=||||||intergenic_region||,WASH7P|E...",ravi
2,chr1,10175,MU23014170,T,A,.,.,"CONSEQUENCE=||||||intergenic_region||,WASH7P|E...",ravi
3,chr1,10181,MU4819728,A,T,.,.,"CONSEQUENCE=||||||intergenic_region||,WASH7P|E...",ravi
4,chr1,10237,MU4630481,A,C,.,.,"CONSEQUENCE=||||||intergenic_region||,WASH7P|E...",ravi


In [5]:
with open('g:/pancan/somatic.txt', 'r') as f:
    vcfs = f.read().split('\n')
vcfs = vcfs[:-1]
print len(vcfs), vcfs[0], vcfs[-1]

2081 g:\pancan2\vcf\0009b464-b376-4fbc-8a56-da538269a02f.svcp_1-0-4.20150204.somatic.snv_mnv.vcf.gz g:\pancan2\vcf\fe8b7893-4682-4a85-a520-8e1d654d037a\1bdfaa8a-1b7e-4251-b135-ce5e4568371b.svcp_1-0-4.20150116.somatic.snv_mnv.vcf.gz


In [6]:
# get a set of TF binding matrices
with open('../regmut/pfm_vertebrates.txt', 'r') as h:
    jaspar = {m.name.strip().split(':')[0].upper(): m.counts.normalize().log_odds() for m in motifs.parse(h, 'jaspar')}

In [7]:
# reduce tfbs to those for which we have matrices
print len(set(tfbs['gene'])), len(set(jaspar.keys())), len(set(tfbs['gene']).intersection(set(jaspar.keys())))
common = set(tfbs['gene']).intersection(set(jaspar.keys()))
tfbs2 = tfbs[tfbs.gene.isin(common)]

161 196 80


In [9]:
def score_mut(m):
    mx = jaspar[m['gene']]
    n = len(mx[mx.keys()[0]])
    p = int(m['pos'])
    refseq = hg19[m['chr']][p-n-1:p+n]
    refscore = reduce(max, (score for pos, score in mx.search(refseq, threshold=0.5) if score > 0 and pos >=0 and pos < n), 0)
    mutseq = refseq[:n] + m['var_allele'] + refseq[n+1:]
    mutscore = reduce(max, (score for pos, score in mx.search(mutseq, threshold=0.5) if score > 0 and pos >=0 and pos < n), 0)
    return (mutscore - refscore) / mx.max

In [8]:
# get reference genome
hg19 = {contig.name: Seq(str(contig.seq), IUPAC.unambiguous_dna)
        for contig in SeqIO.parse('d:/sequence/genomes/hg19.fa', 'fasta')}

In [11]:
# left join by genomic regions
# cols[12] are names of chrom, start, end followed by names of columns to include in join result
# limitations:
# genomic range columns (rcols1, rcols2) must be distinct
# only works if region sets are non-overlapping
# drops duplicates
def merge_regions(df1, cols1, df2, cols2):
    result = None
    [seqname1, start1, end1] = cols1[:3]
    [seqname2, start2, end2] = cols2[:3]
    cols1 = list(pd.Series(cols1).drop_duplicates())
    cols2 = list(pd.Series(cols2).drop_duplicates())
    for contig in set(df1[seqname1]).intersection(set(df2[seqname2])):
        c1 = df1[df1[seqname1]==contig].sort(end1).reset_index(drop=True)
        c2 = df2[df2[seqname2]==contig].sort(start2).drop_duplicates(start2).reset_index(drop=True)
        imerge = pd.cut(c1[end1], c2[start2], labels=False)
        keep = imerge >= 0
        cmerge = pd.concat([c1[cols1][keep], c2[cols2].iloc[imerge[keep]].reset_index(drop=True)], axis=1).reset_index(drop=True)
        cmerge = cmerge[cmerge[start1] <= cmerge[end2]]
        result = cmerge if result is None else pd.concat([result, cmerge])
    return result

In [41]:
muts_tf = merge_regions(muts, ['chr','start','end','var_allele','tumorDNARatio','tumorRNARatio','FractionOfMeanExpression','zOfmeanExpression'],
             tfbs2.rename(columns={'start':'tfstart','end':'tfend'}), ['seqname', 'tfstart', 'tfend', 'gene'])
print muts_tf.describe()
muts_tf[:5]

                start             end  tumorDNARatio  tumorRNARatio  \
count       62.000000       62.000000      62.000000      62.000000   
mean   7550435.612903  7550435.774194       2.239220      85.525777   
std      29657.851182    29657.965000       2.680856     273.122036   
min    7517622.000000  7517622.000000       0.000100       0.000100   
25%    7518239.000000  7518239.000000       0.566770       0.639803   
50%    7576852.000000  7576852.000000       1.175000       3.785632   
75%    7576855.000000  7576855.000000       2.783596      10.854981   
max    7579912.000000  7579912.000000      12.200000    1000.000000   

       FractionOfMeanExpression  zOfmeanExpression         tfstart  \
count                 62.000000          62.000000       62.000000   
mean                   0.680682          -0.524954  7550293.435484   
std                    0.625673           0.984256    29560.420910   
min                    0.098993          -2.095353  7517536.000000   
25%       

,chr,start,end,var_allele,tumorDNARatio,tumorRNARatio,FractionOfMeanExpression,zOfmeanExpression,seqname,tfstart,tfend,gene
5,chr17,7517622,7517622,G,0.857143,0.127660,0.433537,-1.497759,chr17,7517536,7517957,CTCF
6,chr17,7517742,7517742,C,3.426396,11.823529,0.710731,-0.383841,chr17,7517679,7518336,YY1
7,chr17,7517750,7517750,A,4.407407,2.058824,0.152573,-1.124477,chr17,7517679,7518336,YY1
8,chr17,7517783,7517783,C,1.379747,0.761905,0.167333,-1.104892,chr17,7517777,7518033,REST
9,chr17,7517806,7517806,T,8.238095,4.520833,1.778745,2.059043,chr17,7517786,7518620,E2F1


In [42]:
muts_tf['tfscore'] = muts_tf.rename(columns={'start':'pos'}).apply(score_mut, axis=1)

In [44]:
#muts_tf.tfscore.describe()
muts.describe()

,Entrez_Gene_Id,Chromosome,Start_Position,End_Position,Line_Number,n_DNA_Matching_Reference,n_DNA_Matching_Tumor,n_DNA_Matching_Neither,n_DNA_Matching_Both,n_RNA_Matching_Reference,...,tumorRNAFraction,tumorDNAMultiple,tumorRNAMultiple,tumorDNARatio,tumorRNARatio,FractionOfMeanExpression,zOfmeanExpression,IsSingle,start,end
count,2218.000000,2218,2218.000000,2218.000000,2162.000000,2218.000000,2218.000000,2218.000000,2218.000000,2218.000000,...,2218.000000,2218.000000,2218.000000,2218.000000,2218.000000,2215.000000,2215.000000,2218,2218.000000,2218.000000
mean,6308.356628,17,7575980.814247,7575981.141118,53562.034228,54.917042,42.519387,7.951758,5.165464,39.779982,...,0.587831,-4.763100,-0.726035,2.447353,14.845367,1.009202,0.005774,0.8498647,7575980.814247,7575981.141118
std,2314.294669,0,10218.657726,10218.713277,51383.442592,103.786970,68.526194,29.254640,36.295859,57.547324,...,0.321791,23.235304,22.246515,30.123471,101.097243,0.647716,1.044555,0.3572845,10218.657726,10218.713277
min,0.000000,17,7514707.000000,7514707.000000,32.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,-567.000000,-662.000000,0.000100,0.000100,0.073070,-2.806443,False,7514707.000000,7514707.000000
25%,7157.000000,17,7577120.000000,7577120.000000,10781.250000,14.000000,13.000000,2.000000,0.000000,10.000000,...,0.339961,-3.290033,-1.941518,0.303950,0.515062,0.530168,-0.757474,1,7577120.000000,7577120.000000
50%,7157.000000,17,7577574.000000,7577574.000000,36853.000000,29.000000,25.000000,4.000000,0.000000,22.500000,...,0.704791,-1.364510,2.387427,0.732864,2.387427,0.935619,-0.109412,1,7577574.000000,7577574.000000
75%,7157.000000,17,7578406.000000,7578406.000000,79950.750000,56.750000,46.000000,8.000000,0.000000,48.750000,...,0.863636,1.691106,6.333333,1.691106,6.333333,1.324185,0.537331,1,7578406.000000,7578406.000000
max,7157.000000,17,7579912.000000,7579912.000000,314493.000000,1917.000000,1188.000000,909.000000,905.000000,894.000000,...,1.000000,68.000000,136.000000,1000.000000,1000.000000,5.861530,6.189130,True,7579912.000000,7579912.000000


In [4]:
scfiles = pd.read_table('d:/sequence/pancan/santa_cruz_pilot.v2.2015_0504.tsv')
scfiles[:1].transpose()

,0
Project code,BLCA-US
Submitter donor ID,096b4f32-10c1-4737-a0dd-cae04c54ee33
Data train,train2
Train2 pilot,True
Normal WGS submitter specimen ID,27461a27-26eb-4c2c-9c54-e16fbd32c615
Normal WGS submitter sample ID,e0fccaf5-925a-41f9-b87c-cd5ee4aecb59
Normal WGS aliquot ID,e0fccaf5-925a-41f9-b87c-cd5ee4aecb59
Normal WGS alignment GNOS repo(s),https://cghub.ucsc.edu/
Normal WGS alignment GNOS analysis ID,f22a72c5-73c8-478d-b03e-04599b9d5321
Normal WGS alignment BAM file name,PCAWG.67455c36-aa47-4cc4-8b6d-9a9012b616ed.bam


In [5]:
mutsamples = set(muts.Tumor_Sample_UUID)
mutscfiles = scfiles[scfiles['Sanger variant call GNOS file name prefix'].apply(lambda f: f in mutsamples)]
vcfdir = 'g:/pancan2/vcf'
mutscfiles = [vcfdir + '/' + f for f in os.listdir(vcfdir)
              if f.split('.')[0] in set(mutscfiles['Sanger variant call GNOS file name prefix'])]
len(mutscfiles)
'\n'.join(mutscfiles)

'g:/pancan2/vcf/00db4dc2-3ec7-4ff9-9233-d69c8c8a607f.svcp_1-0-5.20150403.somatic.snv_mnv.vcf.gz\ng:/pancan2/vcf/020fab36-c7de-4933-b2bf-dc7b019a1326.svcp_1-0-5.20150324.somatic.snv_mnv.vcf.gz\ng:/pancan2/vcf/0332b017-17d5-4083-8fc4-9d6f8fdbbbde.svcp_1-0-5.20150308.somatic.snv_mnv.vcf.gz\ng:/pancan2/vcf/03ced0ce-186a-4349-8d98-572c2bc90382.svcp_1-0-5.20150401.somatic.snv_mnv.vcf.gz\ng:/pancan2/vcf/04339769-517c-448d-a7ca-951f83608c60.svcp_1-0-5.20150305.somatic.snv_mnv.vcf.gz\ng:/pancan2/vcf/0448206f-3ade-4087-b1a9-4fb2d14e1367.svcp_1-0-5.20150406.somatic.snv_mnv.vcf.gz\ng:/pancan2/vcf/04aa6b77-8074-480c-872e-a1a47afa5314.svcp_1-0-5.20150324.somatic.snv_mnv.vcf.gz\ng:/pancan2/vcf/0980e7fd-051d-45e9-9ca6-2baf073da4e8.svcp_1-0-5.20150306.somatic.snv_mnv.vcf.gz\ng:/pancan2/vcf/09bc8542-4ea1-4599-91b3-d606425b37e2.svcp_1-0-5.20150328.somatic.snv_mnv.vcf.gz\ng:/pancan2/vcf/0bbc7ede-5022-4084-925c-d65baaf7abc2.svcp_1-0-5.20150307.somatic.snv_mnv.vcf.gz\ng:/pancan2/vcf/0c39c03a-ec84-42c6-be69-

In [69]:
from ipyparallel import Client
c = Client()
dview = c[:]
%px import pandas as pd
%px import numpy as np

dview.scatter('vcfs', mutscfiles)
dview.push(dict(read_vcf=read_vcf))
%px variants = [read_vcf(f, somatic=True, donor=f.split('/')[-1].split('.')[0], skipcols=['info', 'format']) for f in vcfs]
variants = dview.gather('variants').get()
variants = pd.concat(variants)
# clean up worker memory
%px variants=None

In [ ]:
muts=p53
mutvars = muts.rename(columns={'Tumor_Sample_UUID':'donor'}).merge(variants, on='donor', how='inner')

In [99]:
print len(mutvars.donor), len(set(mutvars.donor))
mutvars[:1].transpose()

18178112 112


,0
RNAFile,kich\e194e19e-971d-4f34-98d9-1c3c4301ecba-TP53...
DNAFile,kich\14c3ab39-e883-404f-a83c-46733e4a4adc-TP53...
Hugo_Symbol,TP53
Entrez_Gene_Id,7157
Center,broad.mit.edu;hgsc.bcm.edu;bcgsc.ca
NCBI_Build,37
Chromosome,17
Start_Position,7578208
End_Position,7578208
Strand,+


In [85]:
# get enhancer-gene association from Ernst et al
def read_ernst(f):
    df = pd.read_csv(os.path.join('../regmut/data/ernst', f), names=['enhchr','enhstart','enhend','gene_name'], sep='\t')
    df['CellLine'] = f.split('_')[1]
    return df
ernstgene = pd.concat((read_ernst(f) for f in os.listdir('../regmut/data/ernst') if f.startswith('links_') & f.endswith('.txt')))
promgene = ernstgene
promgene[:2]

,enhchr,enhstart,enhend,gene_name,CellLine
0,chr1,2155200,2155400,NM_003036,GM12878
1,chr1,2155400,2155600,NM_003036,GM12878


In [119]:
mut_prom = promgene[promgene.gene_name.isin(['NM_000546'])]
print len(mut_prom.gene_name), set(mut_prom.gene_name)
mut_prom.sort(['enhchr','enhstart','CellLine']).drop_duplicates(['enhchr','enhstart'])

9 set(['NM_000546'])


,enhchr,enhstart,enhend,gene_name,CellLine
5866,chr17,7520200,7520400,NM_000546,Huvec
5868,chr17,7520400,7520600,NM_000546,Huvec
5870,chr17,7520600,7520800,NM_000546,Huvec
5872,chr17,7520800,7521000,NM_000546,Huvec
5874,chr17,7521000,7521200,NM_000546,Huvec
5876,chr17,7521200,7521400,NM_000546,Huvec
5878,chr17,7521400,7521600,NM_000546,Huvec
5855,chr17,7521600,7521800,NM_000546,HSMM


In [129]:
#mutvars['chr'] = mutvars['Chromosome'].apply(lambda c: 'chr' + str(c))
mut_prom_vars = merge_regions(mut_prom.sort(['enhchr','enhstart']), ['enhchr', 'enhstart', 'enhend', 'CellLine', 'gene_name'],
                              mutvars.sort(['chrom','pos']), ['chrom', 'pos', 'pos', 'donor', 'Tumor_Seq_Allele_1',
                                        'Tumor_Seq_Allele_2', 'Variant_Classification', 'Variant_Type',
                                        'tumorDNARatio', 'tumorRNARatio', 'FractionOfMeanExpression', 'zOfmeanExpression'])

In [141]:
def merge_regions(df1, cols1, df2, cols2):
    result = None
    [seqname1, start1, end1] = cols1[:3]
    [seqname2, start2, end2] = cols2[:3]
    cols1 = list(pd.Series(cols1).drop_duplicates())
    cols2 = list(pd.Series(cols2).drop_duplicates())
    df1 = df1[cols1].sort([seqname1, start1, end1])
    df2 = df2[cols2].sort([seqname2, start2, end2])
    for contig in set(df1[seqname1]).intersection(set(df2[seqname2])):
        c1 = df1[df1[seqname1]==contig].sort(end1).reset_index(drop=True)
        c2 = df2[df2[seqname2]==contig].sort(start2).drop_duplicates(start2).reset_index(drop=True)
        imerge = pd.cut(c1[end1], c2[start2], labels=False)
        keep = imerge >= 0
        cmerge = pd.concat([c1[cols1][keep], c2[cols2].iloc[imerge[keep]].reset_index(drop=True)], axis=1).reset_index(drop=True)
        cmerge = cmerge[cmerge[start1] <= cmerge[end2]]
        result = cmerge if result is None else pd.concat([result, cmerge])
    return result

In [142]:
mvp53 = mutvars[(mutvars.chrom=='chr17') & (mutvars.pos >= 7519000) & (mutvars.pos <= 7524000)]
promp53 = mut_prom[(mut_prom.enhchr=='chr17') & (mut_prom.enhstart >= 7519000) & (mut_prom.enhend <= 7524000)]

In [156]:
#merge_regions(promp53, ['enhchr', 'enhstart', 'enhend'], mvp53, ['chrom', 'pos', 'pos'])
#merge_regions(mvp53, ['chrom', 'pos', 'pos'], promp53, ['enhchr', 'enhstart', 'enhend'])
#[df1, seqname1, start1, end1] = [promp53, 'enhchr', 'enhstart', 'enhend']
#[df2, seqname2, start2, end2] = [mvp53, 'chrom', 'pos', 'pos']
#c1 = df1.sort(end1).reset_index(drop=True)
#c2 = df2.sort(start2).drop_duplicates(start2).reset_index(drop=True)
#imerge = pd.cut(c1[end1], c2[start2], labels=False)
keep = imerge >= 0
cmerge = pd.concat([c1[['enhchr', 'enhstart', 'enhend']][keep], 
                    c2[['chrom', 'pos']].iloc[imerge[keep]].reset_index(drop=True)], axis=1).reset_index(drop=True)
cmerge

,enhchr,enhstart,enhend,chrom,pos
0,chr17,7520200,7520400,chr17,7519825
1,chr17,7520400,7520600,chr17,7519825
2,chr17,7520600,7520800,chr17,7519825
3,chr17,7520800,7521000,chr17,7519825
4,chr17,7521000,7521200,chr17,7521110
5,chr17,7521200,7521400,chr17,7521110
6,chr17,7521400,7521600,chr17,7521110
7,chr17,7521600,7521800,chr17,7521788
8,chr17,7521600,7521800,chr17,7521788


In [163]:
# find overlaps between ranges of two dataframes
# return two arrays of indices into dataf
def find_overlaps(df1, start1, end1, df2, start2, end2):
    df1 = df1[[start1, end1]].sort(end1).reset_index(drop=True)
    df2 = df2[[start2, end2]].sort(start2).reset_index(drop=True)
    i1 = i2 = 0
    result1 = []
    result2 = []
    while i1 < len(df1[start1]):
        while 

,chrom,pos
10579583,chr17,7521110
6915730,chr17,7521767
12220598,chr17,7521767
6915731,chr17,7521770
10048648,chr17,7521770
10136707,chr17,7521770
6737809,chr17,7521788
11741822,chr17,7521788


In [159]:
print c1[['enhchr', 'enhstart', 'enhend']], "\n", c2[['chrom', 'pos']], "\n", imerge

  enhchr  enhstart   enhend
0  chr17   7520200  7520400
1  chr17   7520400  7520600
2  chr17   7520600  7520800
3  chr17   7520800  7521000
4  chr17   7521000  7521200
5  chr17   7521200  7521400
6  chr17   7521400  7521600
7  chr17   7521600  7521800
8  chr17   7521600  7521800 
    chrom      pos
0   chr17  7519513
1   chr17  7519769
2   chr17  7519825
3   chr17  7521110
4   chr17  7521767
5   chr17  7521770
6   chr17  7521788
7   chr17  7522135
8   chr17  7522691
9   chr17  7522708
10  chr17  7523029
11  chr17  7523438
12  chr17  7523488 
0    2
1    2
2    2
3    2
4    3
5    3
6    3
7    6
8    6
dtype: int64


In [132]:
print len(mut_prom_vars.donor), len(set(mut_prom_vars.donor))
mut_prom_vars[:5]

3 2


,enhchr,enhstart,enhend,CellLine,gene_name,chrom,pos,donor,Tumor_Seq_Allele_1,Tumor_Seq_Allele_2,Variant_Classification,Variant_Type,tumorDNARatio,tumorRNARatio,FractionOfMeanExpression,zOfmeanExpression
4,chr17,7521000,7521200,Huvec,NM_000546,chr17,7521110,0980e7fd-051d-45e9-9ca6-2baf073da4e8,G,T,Nonsense_Mutation,SNP,0.232558,0.322581,0.195664,-1.617117
7,chr17,7521600,7521800,HSMM,NM_000546,chr17,7521788,c612570a-881d-421f-98b9-0e4563133744,T,C,Missense_Mutation,SNP,0.348837,2.527559,1.240533,0.406890
8,chr17,7521600,7521800,Huvec,NM_000546,chr17,7521788,c612570a-881d-421f-98b9-0e4563133744,T,C,Missense_Mutation,SNP,0.348837,2.527559,1.240533,0.406890


In [134]:
mutvars_p53_enh = mutvars[(mutvars.chrom=='chr17') & (mutvars.pos >= 7520200) & (mutvars.pos <= 7521800)]
mutvars_p53_enh.transpose()

,6737809,6915730,6915731,10048648,10136707,10579583,11741822,12220598
RNAFile,stad\6f62c5a4-97f9-4264-b633-edd368c44fb5-TP53...,brca\12f66fb5-7444-410b-9c93-f275a09ce4dc-TP53...,brca\12f66fb5-7444-410b-9c93-f275a09ce4dc-TP53...,brca\3d40ae3e-c664-4370-95d8-780e760d6d08-TP53...,brca\6e9d893d-fdf1-45b4-b1e0-7142863aafe9-TP53...,coad\192a5602-3bc2-4bcc-8a87-ef039626f092-TP53...,stad\4fc386ae-5646-4041-ac9a-0747d3162d26-TP53...,lusc\548da644-a577-4c86-b04e-2ca21fe4c774-TP53...
DNAFile,stad\ada589d6-c2db-42c0-a927-7ddf0cb3de85-TP53...,brca\8f0ce179-a754-464d-84b6-31f8a1a8f77f-TP53...,brca\8f0ce179-a754-464d-84b6-31f8a1a8f77f-TP53...,brca\f945990a-00d4-4cef-848b-8f145dd27601-TP53...,brca\649630d6-3318-4e29-b003-71980bbde398-TP53...,coad\314d37dc-625e-4f01-87a3-4a6a884bd32f-TP53...,stad\8d796323-4cea-4633-86e8-bc4f3fb0b3c1-TP53...,lusc\1ccbfc36-db90-477a-9333-c733b12ff52c-TP53...
Hugo_Symbol,TP53,TP53,TP53,TP53,TP53,TP53,TP53,TP53
Entrez_Gene_Id,7157,0,0,0,0,7157,7157,7157
Center,broad.mit.edu,genome.wustl.edu,genome.wustl.edu,genome.wustl.edu,genome.wustl.edu,hgsc.bcm.edu,broad.mit.edu,broad.mit.edu
NCBI_Build,37,37,37,37,37,37,37,37
Chromosome,17,17,17,17,17,17,17,17
Start_Position,7578271,7578403,7578403,7578188,7578212,7579406,7578424,7579470
End_Position,7578271,7578403,7578403,7578188,7578212,7579406,7578424,7579470
Strand,+,+,+,+,+,+,+,+


In [115]:
hgnc = pd.read_table('c:/ravip/rstuff/hgnc.tsv')
hgnc[hgnc['Approved Symbol']=='TP53']

,HGNC ID,Approved Symbol,Approved Name,Status,Previous Symbols,Synonyms,Chromosome,Accession Numbers,Entrez Gene ID,Ensembl Gene ID,RefSeq IDs,Vega ID,OMIM ID(supplied by OMIM),UniProt ID(supplied by UniProt)
16964,HGNC:11998,TP53,tumor protein p53,Approved,NaN,"p53, LFS1",17p13.1,AF307851,7157,ENSG00000141510,NM_000546,OTTHUMG00000162125,191170,P04637
